# Variables

Resource: [TensorFlow Guide](https://www.tensorflow.org/guide/variable)

A TensorFlow variable is the recommended way to represent shared, persistent state your program manipulates. This guide covers how to create, update, and manage instances of tf.Variable in TensorFlow.

Variables are created and tracked via the tf.Variable class. A tf.Variable represents a tensor whose value can be changed by running ops on it. Specific ops allow you to read and modify the values of this tensor. Higher level libraries like tf.keras use tf.Variable to store model parameters.

In [1]:
import tensorflow as tf

## Creating Variables

In [2]:
my_tensor = tf.constant([[1, 2.], [3., 4]])
my_variable = tf.Variable(my_tensor)

In [3]:
print(my_tensor)

tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


In [4]:
print(my_variable)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>


In [5]:
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 5 - 2j])

print(bool_variable)
print(complex_variable)

<tf.Variable 'Variable:0' shape=(4,) dtype=bool, numpy=array([False, False, False,  True])>
<tf.Variable 'Variable:0' shape=(2,) dtype=complex128, numpy=array([5.+4.j, 5.-2.j])>


A variable looks and acts like a tensor, and, in fact, is a data structure backed by a tf.Tensor. Like tensors, they have a dtype and a shape, and can be exported to NumPy.

### Variable Properties

In [6]:
print("Shape: \n", my_variable.shape)
print("\nDType: \n", my_variable.dtype)
print("\nAs NumPy: \n", my_variable.numpy())

Shape: 
 (2, 2)

DType: 
 <dtype: 'float32'>

As NumPy: 
 [[1. 2.]
 [3. 4.]]


In [7]:
print("A variable:\n", my_variable)
print("\nViewed as a tensor:\n", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:\n", tf.reduce_sum(my_variable))

# This creates a new tensor; it does not reshape the variable.
print("\nCopying and reshaping: \n", tf.reshape(my_variable, [1,4]))

A variable:
 <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Viewed as a tensor:
 tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

Index of highest value:
 tf.Tensor(10.0, shape=(), dtype=float32)

Copying and reshaping: 
 tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


## Reassignment

As noted above, variables are backed by tensors. You can reassign the tensor using tf.Variable.assign. Calling assign does not (usually) allocate a new tensor; instead, the existing tensor's memory is reused.

In [8]:
a = tf.Variable([2., 3.])
print(a)

a.assign([1, 2])
print(a)

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>


In [9]:
# Not allowed as it resizes the variable:
try:
  a.assign([1.0, 2.0, 3.0])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

ValueError: Cannot assign value to variable ' Variable:0': Shape mismatch.The variable shape (2,), and the assigned value shape (3,) are incompatible.


If you use a variable like a tensor in operations, you will usually operate on the backing tensor.

Creating new variables from existing variables duplicates the backing tensors. Two variables will not share the same memory.

In [10]:
a = tf.Variable([2., 3.])
b = tf.Variable(a)
b.assign([5, 6])

print(a.numpy())
print(b.numpy())

[2. 3.]
[5. 6.]


In [11]:
print(a.numpy())
print(a.assign_add([7, 10]).numpy())
print(a.numpy())

[2. 3.]
[ 9. 13.]
[ 9. 13.]


In [12]:
print(b.numpy())
print(b.assign_sub([2, 8]).numpy())
print(b.numpy())

[5. 6.]
[ 3. -2.]
[ 3. -2.]


## Lifecyclese, Naming, and Watching

In Python-based TensorFlow, tf.Variable instance have the same lifecycle as other Python objects. When there are no references to a variable it is automatically deallocated.

Variables can also be named which can help you track and debug them. You can give two variables the same name.

In [13]:
# Create a and b; they will have the same name but will be backed by
# different tensors.
a = tf.Variable(my_tensor, name="Mark")
# A new variable with the same name, but different value
# Note that the scalar add is broadcast
b = tf.Variable(my_tensor + 1, name="Mark")

# These are elementwise-unequal, despite having the same name
print(a == b)

tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


Although variables are important for differentiation, some variables will not need to be differentiated. You can turn off gradients for a variable by setting trainable to false at creation. An example of a variable that would not need gradients is a training step counter.

In [14]:
step_counter = tf.Variable(1, trainable=False)

## Tensors and Variables Placement

For better performance, TensorFlow will attempt to place tensors and variables on the fastest device compatible with its dtype. This means most variables are placed on a GPU if one is available.

However, you can override this. In this snippet, place a float tensor and a variable on the CPU, even if a GPU is available. By turning on device placement logging (see Setup), you can see where the variable is placed.

In [15]:
with tf.device('CPU:0'):

  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


It's possible to set the location of a variable or tensor on one device and do the computation on another device. This will introduce delay, as data needs to be copied between the devices.

You might do this, however, if you had multiple GPU workers but only want one copy of the variables.

Note: Because tf.config.set_soft_device_placement is turned on by default, even if you run this code on a device without a GPU, it will still run. The multiplication step will happen on the CPU.

In [16]:
with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  # Element-wise multiply
  k = a * b

print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)
